# GEFS Wave Global Collection

Reads the Global GEFS Wave grib collection from NODD's S3 bucket and creates a single dataset using fsspec's ReferenceFileSystem

This notebook demonstrates how to generate the reference JSON files using Kerchunk and gribberish. It was adapted from [this notebook](https://nbviewer.org/gist/peterm790/92eb1df3d58ba41d3411f8a840be2452)

In [1]:
import fsspec

Create the virtual filesystem from which to read the NODD s3 bucket. We are going to write the resulting dataset out to our local filesystem, so we leave the write system blank

In [2]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_ssl=False) # For now SSL false is solving my cert issues **shrug**
fs_write = fsspec.filesystem('')

Now, grab all of the files for the given model run

In [3]:
#https://noaa-gefs-pds.s3.amazonaws.com/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f012.grib2
gefs_ens_member_files = fs_read.glob('s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.*.grib2')

files = sorted(['s3://'+f for f in gefs_ens_member_files])
print(f'Read {len(files)} GEFS files')

Read 105 GEFS files


For each file, we will scan it with gribberish and extract its data to a dictionary formatted to the zarr spec. Then we will write that dictionary to a JSON file with the name formatting specified in the `make_json_name` function

In [4]:
import ujson
from gribberish.kerchunk import scan_gribberish

so = {"anon": True, "use_ssl": False}
json_dir = 'gefs_wave/'

def make_json_name(file_url, message_number): #create a unique name for each reference file
    date = file_url.split('/')[3].split('.')[1]
    name = file_url.split('/')[7].split('.')[0:7]
    return f'{json_dir}{date}_{name[0]}_{name[1]}_{name[2]}_{name[4]}_{name[3]}_{name[6]}_message{message_number}.json'

def gen_json(file_url):
    out = scan_gribberish(file_url, storage_options=so, only_vars=['htsgw'], skip=1) 
    for i, message in enumerate(out):
        print(f'Writing message {i} from {file_url}')
        out_file_name = make_json_name(file_url, i)  # get name
        with fs_write.open(out_file_name, "w") as f: 
            f.write(ujson.dumps(message)) # write to file

In [6]:
%%time
#this step is best run via a cluster
for f in files:
    gen_json(f)

Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f000.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f003.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f006.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f009.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f012.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f015.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f018.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f021.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f024.grib2...
Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gri

Once generated, it can be read back in with xarray or zarr

In [7]:
import xarray as xr

# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='./gefs_wave/20230706_gefs_wave_t12z_global_p01_f000_message0.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False))
ds         

type: [Errno 2] No such file or directory: '/Users/matthewiannucci/Developer/gribberish/python/examples/gefs_wave/20230706_gefs_wave_t12z_global_p01_f000_message0.json/.zmetadata'

In [ ]:
ds.htsgw.plot()

We can also create a single dataset from the collection of the kerchunked files 

In [ ]:
from kerchunk.combine import MultiZarrToZarr
import numpy as np

reference_jsons = fs_write.ls(json_dir) #get list of file names

def fn_to_ens(index, fs, var, fn):
    import re
    import datetime
    groups = re.search(r"\_p(\d{2})", fn).groups()
    return int(groups[0])

# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(reference_jsons,
                        coo_map={'ens': fn_to_ens},
                        coo_dtypes={'ens': np.dtype('int32')},
                        concat_dims = ['ens', 'time'],
                        identical_dims=['latitude', 'longitude'])
d = mzz.translate()

with open('./gefs_wave/gefswave_kerchunk.json', 'w') as f:
    f.write(ujson.dumps(d))

In [ ]:
# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='./gefs_wave/gefswave_kerchunk.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'time':1})

ds                      